<a href="https://colab.research.google.com/github/janithcyapa/Decentralized-HVAC-Control-System/blob/main/energy-plus-utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Usage of Energy Plus Utility**

## **Installation**

In [1]:
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()  # raises on failure, otherwise silent
# TODO: Allow user to specify EPLUS_DIR when running the prepare_colab_eplus()

In [4]:
from eplus import EPlusUtil, EPlusSqlExplorer
import subprocess, json, pathlib, os
import pandas as pd
import json

## **Initialization**

### **Selection of IDF files from examples**

#### **Initial implementation**
`1ZoneDataCenterCRAC_wPumpedDXCoolingCoil.idf`
- Single-zone data center
- CRAC unit (Computer Room Air Conditioner)
- Simple, stable HVAC topology
- Real HVAC system
- Clear air mass flow path
- Clear supply air temperature control

Or use
`1ZoneUncontrolled.idf + ZoneHVAC:IdealLoadsAirSystem`
- Fast, conceptual, controller-logic sandbox
- Single thermal zone
- Simple envelope & loads
- Infinite heating/cooling capacity
- Zone temperature/humidity control


#### **Decentralized Controlling**
`5ZoneAirCooledWithDOASAirLoop.idf`.
- DOAS (Dedicated Outdoor Air System)
- Separation of ventilation & conditioning
- Multi-zone building
- Central AHU
- Outdoor air handling
- Zone-level terminals
- Mass flow modulation
- Ventilation control logic

Implementation
- Decentralized zone controller - VAV / terminal unit control
- Zone mass flow rate command - AirTerminal:SingleDuct:*
- Central AHU coordinator - AirLoopHVAC
- AHU supply air temperature - Setpoint Manager (SAT)
- AHU humidity ratio - Humidity control on air loop
- CO₂ coordination - Controller:MechanicalVentilation
- Aggregate ventilation logic - Outdoor Air Controller

As a Secondary / fallback try `5ZoneAirCooled.idf`
- Multiple zones
- Central air system
- Coils, fans, schedules

In [68]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"
OUT_DIR = "/content/eplus_out"

# Define the Simulation Model
# IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
IDF = f"{EPLUS_DIR}/ExampleFiles/1ZoneDataCenterCRAC_wPumpedDXCoolingCoil.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"


In [69]:
!rm -rf /content/eplus_out/*

In [76]:
# Create utility instance and set simulation model
util = EPlusUtil(verbose=1, out_dir = OUT_DIR)
# Reset and set model
util.delete_out_dir()
util.set_model(idf = IDF, epw = EPW,add_co2=False)
util.ensure_output_sqlite()
util.enable_runtime_logging()

### **Visualisation of IDF files**

In [77]:
# Convert the model to .json
idf_path = pathlib.Path(IDF)
converter = os.path.join(EPLUS_DIR, "ConvertInputFormat")   # on Windows it's ConvertInputFormat.exe
subprocess.run([converter, str(idf_path)], check=True)
epjson_path = idf_path.with_suffix(".epJSON")

In [72]:
# @title
def list_epjson_tables(epjson: dict):
    rows = []
    for obj_type, objs in epjson.items():
        count = len(objs) if isinstance(objs, dict) else "-"
        rows.append((obj_type, count))

    print(f"{'Object Type':45} | {'# Objects'}")
    print("-" * 60)
    for r in sorted(rows):
        print(f"{r[0]:45} | {r[1]}")

def show_object_table(epjson: dict, object_type: str):
    table = epjson.get(object_type)
    if table is None:
        print(f"{object_type} not found")
        return

    print(f"{object_type}")
    print("-" * len(object_type))
    for name, fields in table.items():
        print(f"\n{name}")
        for k, v in fields.items():
            print(f"  {k:35} : {v}")
def find_mass_flow_objects(epjson: dict):
    keywords = ["air_flow", "mass_flow", "flow_rate"]
    rows = []

    for obj_type, objs in epjson.items():
        if not isinstance(objs, dict):
            continue
        for name, fields in objs.items():
            for f in fields:
                if any(k in f.lower() for k in keywords):
                    rows.append((obj_type, name, f, fields[f]))

    print(f"{'Object Type':35} | {'Name':25} | {'Field':30} | Value")
    print("-" * 110)
    for r in rows:
        print(f"{r[0]:35} | {r[1]:25} | {r[2]:30} | {r[3]}")

In [73]:
with open(str(epjson_path), "r", encoding="utf-8") as f:
    epjson = json.load(f)
#
# list_epjson_tables(epjson)
# show_object_table(epjson, "AirLoopHVAC")
find_mass_flow_objects(epjson)

Object Type                         | Name                      | Field                          | Value
--------------------------------------------------------------------------------------------------------------
AirLoopHVAC                         | CRAC system               | design_supply_air_flow_rate    | 8.5
AirTerminal:SingleDuct:VAV:NoReheat | Main Zone VAV Air         | constant_minimum_air_flow_fraction | 0.05
AirTerminal:SingleDuct:VAV:NoReheat | Main Zone VAV Air         | maximum_air_flow_rate          | 8.5
AirTerminal:SingleDuct:VAV:NoReheat | Main Zone VAV Air         | zone_minimum_air_flow_input_method | Constant
Coil:Cooling:DX:SingleSpeed         | Main Cooling Coil 1       | 2023_rated_evaporator_fan_power_per_volume_flow_rate | 934.4
Coil:Cooling:DX:SingleSpeed         | Main Cooling Coil 1       | rated_air_flow_rate            | 8.5
ElectricEquipment:ITE:AirCooled     | Data Center Servers       | air_flow_calculation_method    | FlowFromSystem
ElectricEquipm

## **Simulations**

### Dry run



In [78]:
util.dry_run_min(include_ems_edd=True)

EnergyPlus Starting
EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.19 05:50
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Initializing Response Factors
Calculating CTFs for "FLOOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior Conve

0

In [79]:
err_path = f"{OUT_DIR}/eplusout.err"
print(open(err_path, "r", errors="ignore").read()[-4000:])

Program Version,EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.19 05:50,
   ** Warning ** Weather file location will be used rather than entered (IDF) Location object.
   **   ~~~   ** ..Location object=DENVER CENTENNIAL CO USA WMO=724666
   **   ~~~   ** ..Weather File Location=San Francisco Intl Ap CA USA TMY3 WMO#=724940
   **   ~~~   ** ..due to location differences, Latitude difference=[2.12] degrees, Longitude difference=[17.22] degrees.
   **   ~~~   ** ..Time Zone difference=[1.0] hour(s), Elevation difference=[99.89] percent, [1791.00] meters.
   ** Warning ** SetUpDesignDay: Entered DesignDay Barometric Pressure=81560 differs by more than 10% from Standard Barometric Pressure=101301.
   **   ~~~   ** ...occurs in DesignDay=DENVER CENTENNIAL ANN HTG 99.6% CONDNS DB, Standard Pressure (based on elevation) will be used.
   ** Warning ** SetUpDesignDay: Entered DesignDay Barometric Pressure=81560 differs by more than 10% from Standard Barometric Pressure=101301.
   **   ~~~  

In [80]:
util.list_zone_names()

['Main Zone']

In [81]:
util.list_available_variables()

,Kind,VariableName,KeyValue,Units
0,Outputvariable,Site Outdoor Air Drybulb Temperature,ENVIRONMENT,C
1,Outputvariable,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT,C
2,Outputvariable,Site Outdoor Air Wetbulb Temperature,ENVIRONMENT,C
3,Outputvariable,Site Outdoor Air Humidity Ratio,ENVIRONMENT,kgWater/kgDryAir
4,Outputvariable,Site Outdoor Air Relative Humidity,ENVIRONMENT,%
...,...,...,...,...
142,Outputvariable,System Node Setpoint Temperature,MAIN ZONE ATU IN NODE,C
143,Outputvariable,System Node Standard Density Volume Flow Rate,MAIN ZONE ATU IN NODE,m3/s
144,Outputvariable,Environmental Impact Total N2O Emissions Carbo...,SITE,kg
145,Outputvariable,Environmental Impact Total CH4 Emissions Carbo...,SITE,kg


In [44]:
util.list_available_actuators()


,Kind,ComponentType,ControlType,ActuatorKey,Units
0,Actuator,Schedule:Compact,Schedule Value,OCCUPY-1,[ ]
1,Actuator,Schedule:Compact,Schedule Value,LIGHTS-1,[ ]
2,Actuator,Schedule:Compact,Schedule Value,EQUIP-1,[ ]
3,Actuator,Schedule:Compact,Schedule Value,INFIL-SCH,[ ]
4,Actuator,Schedule:Compact,Schedule Value,ACTSCHD,[ ]
...,...,...,...,...,...
1209,Actuator,Outdoor Air System Node,Drybulb Temperature,CENTRAL CHILLER CONDENSER INLET NODE,[C]
1210,Actuator,Outdoor Air System Node,Wetbulb Temperature,CENTRAL CHILLER CONDENSER INLET NODE,[C]
1211,Actuator,Outdoor Air System Node,Wind Speed,CENTRAL CHILLER CONDENSER INLET NODE,[m/s]
1212,Actuator,Outdoor Air System Node,Wind Direction,CENTRAL CHILLER CONDENSER INLET NODE,[degree]


In [ ]:
util.list_available_meters()

,Kind,MeterName,Units


### Config Simulatin

In [90]:
specs = [
    # --- Zone state + people ---
    {"name": "Zone Mean Air Temperature",                "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Dewpoint Temperature",       "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Air Relative Humidity",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Humidity Ratio",             "key": "*",            "freq": "TimeStep"},
    {"name": "Zone People Occupant Count",               "key": "*",            "freq": "TimeStep"},

    # --- CO₂ & OA into zones ---
    {"name": "Zone Air CO2 Concentration",               "key": "*",            "freq": "TimeStep"},

    # --- Site weather (Environment key) ---
    {"name": "Site Outdoor Air Drybulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Wetbulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Dewpoint Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Relative Humidity",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Humidity Ratio",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Barometric Pressure",        "key": "Environment", "freq": "TimeStep"},
    {"name": "Site Outdoor Air CO2 Concentration",                          "key": "Environment",  "freq": "TimeStep"},

    {"name": "System Node Temperature",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Humidity Ratio",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node CO2 Concentration",               "key": "*",            "freq": "TimeStep"},
]

# 1) Ensure the Output:Variable objects exist (dedup-aware)
util.ensure_output_variables(specs, activate=True)


# 2) Ensure the meter(s) you want are reported
output_meters = [
    "Electricity:Facility",
    "ElectricityPurchased:Facility",
    "ElectricitySurplusSold:Facility",
    "Cooling:EnergyTransfer",
    # NOTE: The two below are model-specific. Keep them only if they exist in your IDF.
    # "InteriorLights:Electricity:Zone:SPACE5-1",
    # "Cooling:EnergyTransfer:Zone:SPACE1-1",
]
util.ensure_output_meters(output_meters, freq="TimeStep")


#3) Register callbacks
util.register_handlers(
    "before_hvac",
    [{
        "method_name": "tick_set_actuator",
        "kwargs": {
            "component_type": "Schedule:Compact",      # sometimes "Schedule:Constant" etc.
            "control_type":   "Schedule Value",
            "actuator_key":   "Cooling Return Air Setpoint Schedule",
            "value":          25.0,
            "when":           "success",
            "read_back":      True,
            "precision":      2
        }
    }],
    clear=True,
    run_during_warmup=False
)

rc=util.run_annual()

EnergyPlus Starting
EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.19 06:05
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Initializing Response Factors
Calculating CTFs for "FLOOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior Conve

In [88]:
util.list_available_variables()

,Kind,VariableName,KeyValue,Units
0,Outputvariable,Site Outdoor Air Drybulb Temperature,ENVIRONMENT,C
1,Outputvariable,Site Outdoor Air Drybulb Temperature,ENVIRONMENT,C
2,Outputvariable,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT,C
3,Outputvariable,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT,C
4,Outputvariable,Site Outdoor Air Wetbulb Temperature,ENVIRONMENT,C
...,...,...,...,...
190,Outputvariable,System Node Setpoint Temperature,MAIN ZONE ATU IN NODE,C
191,Outputvariable,System Node Standard Density Volume Flow Rate,MAIN ZONE ATU IN NODE,m3/s
192,Outputvariable,Environmental Impact Total N2O Emissions Carbo...,SITE,kg
193,Outputvariable,Environmental Impact Total CH4 Emissions Carbo...,SITE,kg


In [84]:
xp = EPlusSqlExplorer(f"{OUT_DIR}/eplusout.sql")
xp.list_tables()
df = xp.list_sql_variables(name="System Node Temperature")
df[['KeyValue','n_rows']].head(20)

,KeyValue,n_rows
0,MAIN COOLING COIL 1 OUTLET NODE,288
1,MAIN ZONE ATU IN NODE,288
2,MAIN ZONE INLET NODE,288
3,MAIN ZONE NODE,288
4,MAIN ZONE OUTLET NODE,288
5,SUPPLY INLET NODE,288
6,SUPPLY OUTLET NODE,288
7,ZONE EQUIPMENT INLET NODE,288
8,ZONE EQUIPMENT OUTLET NODE,288
9,MAIN COOLING COIL 1 OUTLET NODE,48


In [91]:
# Drybulb (auto-picks top zone keys if keys=None)
fig1=util.plot_sql_zone_variable(
    "Zone Mean Air Temperature",
    keys=["*"],                          # auto-pick a few zones with data
    reporting_freq=("TimeStep",),       # match how you logged
    resample="1h",
    title="Zone Mean Air Temperature"
)

# Humidity ratio
fig2=util.plot_sql_zone_variable(
    "Zone Mean Air Humidity Ratio",
    keys=["*"],
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Mean Air Humidity Ratio"
)

# CO2 concentration
# fig3=util.plot_sql_zone_variable(
#     "Zone Air CO2 Concentration",
#     keys={"*"},
#     reporting_freq=("TimeStep",),
#     resample="1h",
#     title="Zone Air CO2 Concentration"
# )